In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import os
import random
import time

# --- 配置部分 ---
data_path = r"C:\Users\Michael Wang\OneDrive\小论文\毕业论文改写\WGAN-GP\建模_数据预处理\data\development_set_selected_features.xlsx"
target_column_name = 'Rowing distance'
output_plot_path = r"C:\Users\Michael Wang\OneDrive\小论文\毕业论文改写\WGAN-GP\插图"
os.makedirs(output_plot_path, exist_ok=True)

# 设置随机种子以保证结果可复现
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# 自动选择设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- 数据加载 ---
original_data = pd.read_excel(data_path)
X_original_df = original_data.drop(columns=[target_column_name])
y_original_series = original_data[target_column_name]

# --- BPNN/MLP 模型定义 ---
class BPNN(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim=1, dropout_rate=0.2):
        super(BPNN, self).__init__()
        layers = []
        current_dim = input_dim
        for h_dim in hidden_dims:
            layers.append(nn.Linear(current_dim, h_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate)) # Dropout for regularization
            current_dim = h_dim
        layers.append(nn.Linear(current_dim, output_dim))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

# --- 训练和评估函数 ---
def train_evaluate_fold(model_config, X_train_fold, y_train_fold, X_val_fold, y_val_fold,
                        y_scaler_fold, n_epochs, patience=10):
    """
    训练和评估模型在一个特定的折上。
    """
    input_dim = X_train_fold.shape[1]
    model = BPNN(input_dim, model_config['hidden_dims'], dropout_rate=model_config['dropout_rate']).to(device)
    optimizer = optim.Adam(model.parameters(), lr=model_config['learning_rate'], weight_decay=model_config.get('weight_decay', 0)) # L2正则化
    criterion = nn.MSELoss() # 使用MSELoss进行训练，但我们也会监控MAE

    # 数据加载器
    train_dataset = TensorDataset(torch.FloatTensor(X_train_fold).to(device), torch.FloatTensor(y_train_fold).reshape(-1,1).to(device))
    train_loader = DataLoader(train_dataset, batch_size=model_config['batch_size'], shuffle=True)
    
    val_dataset = TensorDataset(torch.FloatTensor(X_val_fold).to(device), torch.FloatTensor(y_val_fold).reshape(-1,1).to(device))
    val_loader = DataLoader(val_dataset, batch_size=model_config['batch_size'], shuffle=False)

    epoch_train_mae_unnormalized = []
    epoch_val_mae_unnormalized = []
    
    best_val_loss_mae = float('inf')
    epochs_no_improve = 0

    for epoch in range(n_epochs):
        model.train()
        fold_train_preds_unnorm = []
        fold_train_targets_unnorm = []
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

            # For unnormalized MAE tracking during training
            batch_preds_unnorm = y_scaler_fold.inverse_transform(outputs.detach().cpu().numpy())
            batch_targets_unnorm = y_scaler_fold.inverse_transform(batch_y.detach().cpu().numpy())
            fold_train_preds_unnorm.extend(batch_preds_unnorm.flatten())
            fold_train_targets_unnorm.extend(batch_targets_unnorm.flatten())

        current_train_mae_unnorm = mean_absolute_error(fold_train_targets_unnorm, fold_train_preds_unnorm)
        epoch_train_mae_unnormalized.append(current_train_mae_unnorm)

        model.eval()
        fold_val_preds_unnorm = []
        fold_val_targets_unnorm = []
        with torch.no_grad():
            for batch_X_val, batch_y_val in val_loader:
                val_outputs = model(batch_X_val)
                # For unnormalized MAE tracking during validation
                batch_val_preds_unnorm = y_scaler_fold.inverse_transform(val_outputs.cpu().numpy())
                batch_val_targets_unnorm = y_scaler_fold.inverse_transform(batch_y_val.cpu().numpy())
                fold_val_preds_unnorm.extend(batch_val_preds_unnorm.flatten())
                fold_val_targets_unnorm.extend(batch_val_targets_unnorm.flatten())
        
        current_val_mae_unnorm = mean_absolute_error(fold_val_targets_unnorm, fold_val_preds_unnorm)
        epoch_val_mae_unnormalized.append(current_val_mae_unnorm)
        
        # Early stopping based on unnormalized validation MAE
        if current_val_mae_unnorm < best_val_loss_mae:
            best_val_loss_mae = current_val_mae_unnorm
            epochs_no_improve = 0
            # torch.save(model.state_dict(), 'best_model_fold.pth') # Optional: save best model for this fold
        else:
            epochs_no_improve += 1
        
        if epochs_no_improve >= patience:
            # print(f"Early stopping at epoch {epoch+1} for this fold.")
            break
            
    # model.load_state_dict(torch.load('best_model_fold.pth')) # Load best model for evaluation

    # Final evaluation on validation set for this fold (unnormalized)
    model.eval()
    all_y_pred_val_scaled = []
    all_y_val_scaled = []
    with torch.no_grad():
        for batch_X_val, batch_y_val in val_loader:
            val_outputs = model(batch_X_val)
            all_y_pred_val_scaled.extend(val_outputs.cpu().numpy())
            all_y_val_scaled.extend(batch_y_val.cpu().numpy())
            
    y_pred_val_unnorm = y_scaler_fold.inverse_transform(np.array(all_y_pred_val_scaled)).flatten()
    y_val_unnorm = y_scaler_fold.inverse_transform(np.array(all_y_val_scaled)).flatten()

    mae_val = mean_absolute_error(y_val_unnorm, y_pred_val_unnorm)
    mse_val = mean_squared_error(y_val_unnorm, y_pred_val_unnorm)
    rmse_val = np.sqrt(mse_val)
    r2_val = r2_score(y_val_unnorm, y_pred_val_unnorm)

    # Evaluation on training set (for checking overfit, unnormalized)
    all_y_pred_train_scaled = []
    all_y_train_scaled = []
    with torch.no_grad():
        for batch_X_train, batch_y_train in train_loader: # Re-iterate train_loader
            train_outputs = model(batch_X_train)
            all_y_pred_train_scaled.extend(train_outputs.cpu().numpy())
            all_y_train_scaled.extend(batch_y_train.cpu().numpy())

    y_pred_train_unnorm = y_scaler_fold.inverse_transform(np.array(all_y_pred_train_scaled)).flatten()
    y_train_unnorm = y_scaler_fold.inverse_transform(np.array(all_y_train_scaled)).flatten()
    
    mae_train = mean_absolute_error(y_train_unnorm, y_pred_train_unnorm)
    mse_train = mean_squared_error(y_train_unnorm, y_pred_train_unnorm)
    rmse_train = np.sqrt(mse_train)
    r2_train = r2_score(y_train_unnorm, y_pred_train_unnorm)

    return {
        'mae_val': mae_val, 'mse_val': mse_val, 'rmse_val': rmse_val, 'r2_val': r2_val,
        'mae_train': mae_train, 'mse_train': mse_train, 'rmse_train': rmse_train, 'r2_train': r2_train,
        'train_loss_curve': epoch_train_mae_unnormalized,
        'val_loss_curve': epoch_val_mae_unnormalized
    }


# --- 超参数定义 ---
# 示例参数网格，可以根据需要调整和扩展
param_grid = {
    'learning_rate': [0.0005, 0.001, 0.005], # 保持或尝试略小的学习率
    'hidden_dims': [
        [32],                # 非常简单的单层网络
        [64],
        [32, 16],            # 两层，但更简单
        [64, 32],            # 比之前最佳的略简单
        [128, 64, 32]      # 可以保留之前的最佳结构，看增强正则化是否有帮助，或者暂时去掉以强制探索更简单的模型
    ],
    'batch_size': [16, 32, 64], # 之前最佳是32，可以尝试更大的batch_size看是否能稳定训练或有更好的泛化
    'dropout_rate': [0.2, 0.3, 0.4, 0.5], # 显著提高dropout的候选值
    'weight_decay': [1e-4, 5e-4, 1e-3, 2e-3] # 尝试引入并增加L2正则化的强度
}


n_hyperparam_trials = 20 # 随机选择20组超参数进行尝试，可以增加以获得更好的结果
n_epochs_hyperparam_tuning = 50 # 用于超参数调整的epoch数，可以减少以加速搜索
n_epochs_final_training = 200 # 用于最终模型训练的epoch数
patience_hyperparam = 5      # 早停轮数 - 超参数搜索
patience_final = 15          # 早停轮数 - 最终模型

# --- 超参数调优 (类似RandomizedSearchCV) ---
print("开始超参数调优...")
best_params = None
best_avg_val_mae = float('inf') # 我们以验证集上的平均MAE作为评价标准
hyperparam_search_results = []

# Convert DataFrame to NumPy arrays for efficiency before loops
X_np = X_original_df.values
y_np = y_original_series.values

# 尝试 n_hyperparam_trials 组随机参数
sampled_configs = []
for _ in range(n_hyperparam_trials):
    config = {}
    for key, values in param_grid.items():
        config[key] = random.choice(values)
    # 避免重复配置 (可选,对于大量试验可能不必要)
    if config not in sampled_configs:
        sampled_configs.append(config)

print(f"将尝试 {len(sampled_configs)} 组独特的随机超参数组合。")

for i, current_params in enumerate(sampled_configs):
    print(f"\n--- 超参数组合 {i+1}/{len(sampled_configs)} ---")
    print(current_params)
    
    kf_hyper = KFold(n_splits=3, shuffle=True, random_state=42) # 使用3折CV加速超参数搜索
    fold_val_maes = []

    for fold_idx, (train_idx, val_idx) in enumerate(kf_hyper.split(X_np, y_np)):
        X_train_fold, X_val_fold = X_np[train_idx], X_np[val_idx]
        y_train_fold, y_val_fold = y_np[train_idx], y_np[val_idx]

        # 数据归一化 (每折独立，严格来说fit应该只在训练集上，但对于超参数搜索阶段，为简化可以对当前折的X,y都fit_transform)
        # 更严格：fit_transform训练集，transform验证集
        x_scaler_fold = StandardScaler()
        X_train_fold_scaled = x_scaler_fold.fit_transform(X_train_fold)
        X_val_fold_scaled = x_scaler_fold.transform(X_val_fold)

        y_scaler_fold = StandardScaler() # 用于反归一化输出
        y_train_fold_scaled = y_scaler_fold.fit_transform(y_train_fold.reshape(-1, 1)).flatten()
        y_val_fold_scaled = y_scaler_fold.transform(y_val_fold.reshape(-1, 1)).flatten()
        
        # 训练和评估
        fold_results = train_evaluate_fold(
            current_params, X_train_fold_scaled, y_train_fold_scaled,
            X_val_fold_scaled, y_val_fold_scaled, y_scaler_fold, # 传递y_scaler用于反归一化
            n_epochs=n_epochs_hyperparam_tuning, patience=patience_hyperparam
        )
        fold_val_maes.append(fold_results['mae_val'])
        # print(f"  Fold {fold_idx+1} Val MAE: {fold_results['mae_val']:.4f}")

    avg_fold_val_mae = np.mean(fold_val_maes)
    hyperparam_search_results.append({'params': current_params, 'avg_val_mae': avg_fold_val_mae})
    print(f"参数组 {i+1} 平均验证 MAE: {avg_fold_val_mae:.4f}")

    if avg_fold_val_mae < best_avg_val_mae:
        best_avg_val_mae = avg_fold_val_mae
        best_params = current_params

print("\n--- 超参数调优完成 ---")
if best_params:
    print(f"最佳参数: {best_params}")
    print(f"最佳平均验证 MAE: {best_avg_val_mae:.4f}")
else:
    print("未能找到最佳参数，请检查超参数网格或增加试验次数。将使用第一组参数。")
    best_params = sampled_configs[0]


# --- 使用最佳参数进行K折交叉验证评估 (更严格的评估) ---
print("\n--- 使用最佳参数进行K折交叉验证评估 ---")
kf_final = KFold(n_splits=5, shuffle=True, random_state=123) # 最终评估使用5折
final_fold_metrics = []
all_folds_train_loss_curves = []
all_folds_val_loss_curves = []

for fold_num, (train_index, test_index) in enumerate(kf_final.split(X_np, y_np)):
    print(f"\nFold {fold_num + 1}/5")
    X_train, X_test = X_np[train_index], X_np[test_index]
    y_train, y_test = y_np[train_index], y_np[test_index]

    # 数据归一化: Fit on training data, transform both train and test
    x_scaler = StandardScaler()
    X_train_scaled = x_scaler.fit_transform(X_train)
    X_test_scaled = x_scaler.transform(X_test)

    y_scaler = StandardScaler() # 用于反归一化目标值
    y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1)).flatten()
    y_test_scaled = y_scaler.transform(y_test.reshape(-1, 1)).flatten() # 验证集也用训练集fit的scaler

    fold_results = train_evaluate_fold(
        best_params, X_train_scaled, y_train_scaled,
        X_test_scaled, y_test_scaled, y_scaler, # 传递y_scaler
        n_epochs=n_epochs_final_training, patience=patience_final
    )
    
    final_fold_metrics.append(fold_results)
    all_folds_train_loss_curves.append(fold_results['train_loss_curve'])
    all_folds_val_loss_curves.append(fold_results['val_loss_curve'])
    
    print(f"  Train MAE: {fold_results['mae_train']:.4f}, Train R2: {fold_results['r2_train']:.4f}")
    print(f"  Test MAE: {fold_results['mae_val']:.4f}, Test R2: {fold_results['r2_val']:.4f}")


# --- 计算平均性能 ---
avg_metrics = {
    'mae_train': np.mean([m['mae_train'] for m in final_fold_metrics]),
    'mse_train': np.mean([m['mse_train'] for m in final_fold_metrics]),
    'rmse_train': np.mean([m['rmse_train'] for m in final_fold_metrics]),
    'r2_train': np.mean([m['r2_train'] for m in final_fold_metrics]),
    'mae_test': np.mean([m['mae_val'] for m in final_fold_metrics]),
    'mse_test': np.mean([m['mse_val'] for m in final_fold_metrics]),
    'rmse_test': np.mean([m['rmse_val'] for m in final_fold_metrics]),
    'r2_test': np.mean([m['r2_val'] for m in final_fold_metrics]),
}

print("\nBPNN - Average Train Performance (Unnormalized):")
print(f"  MAE: {avg_metrics['mae_train']:.4f}")
print(f"  MSE: {avg_metrics['mse_train']:.4f}")
print(f"  RMSE: {avg_metrics['rmse_train']:.4f}")
print(f"  R2 Score: {avg_metrics['r2_train']:.4f}")

print("\nBPNN - Average Test Performance (Unnormalized):")
print(f"  MAE: {avg_metrics['mae_test']:.4f}")
print(f"  MSE: {avg_metrics['mse_test']:.4f}")
print(f"  RMSE: {avg_metrics['rmse_test']:.4f}")
print(f"  R2 Score: {avg_metrics['r2_test']:.4f}")


# --- 绘制图表并导出 ---

# 1. 绘制评估指标图 (MAE 和 R2)
metrics_plot_names_en = ['MAE', 'R2 Score']
values_train_plot = [avg_metrics['mae_train'], avg_metrics['r2_train']]
values_test_plot = [avg_metrics['mae_test'], avg_metrics['r2_test']]

x_axis_plot = np.arange(len(metrics_plot_names_en))
plt.figure(figsize=(10, 6))
plt.bar(x_axis_plot - 0.2, values_train_plot, width=0.4, label='Train', align='center')
plt.bar(x_axis_plot + 0.2, values_test_plot, width=0.4, label='Test', align='center')
plt.xticks(x_axis_plot, metrics_plot_names_en)
plt.ylabel('Score')
plt.title('BPNN Average Train vs. Test Set Evaluation Metrics')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plot_filename_metrics = os.path.join(output_plot_path, "bpnn_average_evaluation_metrics.png")
plt.savefig(plot_filename_metrics, dpi=300, bbox_inches='tight')
plt.show()


# 2. 绘制每折的训练和测试损失 (MAE, 反归一化)
plt.figure(figsize=(14, 7))
max_epochs_ran = 0
for i in range(len(all_folds_train_loss_curves)):
    # 由于早停，各折的epoch数可能不同
    epochs_this_fold = len(all_folds_train_loss_curves[i])
    max_epochs_ran = max(max_epochs_ran, epochs_this_fold)
    plt.plot(range(1, epochs_this_fold + 1), all_folds_train_loss_curves[i], label=f'Train Fold {i+1}', linestyle='-')
    plt.plot(range(1, epochs_this_fold + 1), all_folds_val_loss_curves[i], label=f'Test Fold {i+1}', linestyle='--')

plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error (MAE) - Unnormalized')
plt.title('BPNN Training and Testing MAE (Unnormalized) per Fold')
plt.xlim(1, max_epochs_ran) # 统一X轴范围
plt.legend(loc='upper right')
plt.grid(True, linestyle='--', alpha=0.7)
plot_filename_loss = os.path.join(output_plot_path, "bpnn_mae_loss_per_fold_unnormalized.png")
plt.savefig(plot_filename_loss, dpi=300, bbox_inches='tight')
plt.show()

print(f"\n所有图表已尝试保存至: {output_plot_path}")

# --- 过拟合检查 (与XGBoost类似) ---
r2_diff = avg_metrics['r2_train'] - avg_metrics['r2_test']
mae_train_avg = avg_metrics['mae_train']
mae_test_avg = avg_metrics['mae_test']

print("\nOverfitting Check:")
if r2_diff > 0.1 and avg_metrics['r2_train'] > avg_metrics['r2_test']:
    print(f"Warning: Model might be overfitting! Train R2 ({avg_metrics['r2_train']:.2f}) is significantly higher than Test R2 ({avg_metrics['r2_test']:.2f}). Difference: {r2_diff:.2f}")
elif mae_train_avg > 0 and (mae_test_avg - mae_train_avg) / mae_train_avg > 0.20 :
     print(f"Warning: Model might be overfitting! Test MAE ({mae_test_avg:.2f}) is >20% higher than Train MAE ({mae_train_avg:.2f}). Relative difference: {((mae_test_avg - mae_train_avg) / mae_train_avg)*100:.2f}%.")
else:
    print(f"No strong signs of overfitting detected based on current thresholds. R2 Diff (Train-Test): {r2_diff:.2f}. MAE Train: {mae_train_avg:.2f}, MAE Test: {mae_test_avg:.2f}.")

In [ ]:
# 基于原始数据的MLP/BPNN模型 (已添加最终测试集评估)
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold, train_test_split # <--- 已在此处添加 train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import os
import random
import time
import joblib

# --- 配置部分 ---
# 开发集路径 (80%的数据)
development_data_path = r"C:\Users\Michael Wang\OneDrive\小论文\毕业论文改写\WGAN-GP\建模_数据预处理\data\development_set_selected_features.xlsx"
# 最终测试集路径 (20%的数据)
final_test_data_path = r"C:\Users\Michael Wang\OneDrive\小论文\毕业论文改写\WGAN-GP\建模_数据预处理\data\final_test_set_selected_features.xlsx"

target_column_name = 'Rowing distance'
output_plot_path = r"C:\Users\Michael Wang\OneDrive\小论文\毕业论文改写\WGAN-GP\插图"
os.makedirs(output_plot_path, exist_ok=True)

# 设置随机种子以保证结果可复现
def set_seed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# 自动选择设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- 数据加载 ---
original_data = pd.read_excel(development_data_path)
X_original_df = original_data.drop(columns=[target_column_name])
y_original_series = original_data[target_column_name]
print(f"开发集数据加载成功，形状: {original_data.shape}")

# --- BPNN/MLP 模型定义 ---
class BPNN(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim=1, dropout_rate=0.2):
        super(BPNN, self).__init__()
        layers = []
        current_dim = input_dim
        for h_dim in hidden_dims:
            layers.append(nn.Linear(current_dim, h_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            current_dim = h_dim
        layers.append(nn.Linear(current_dim, output_dim))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

# --- 训练和评估函数 ---
def train_evaluate_fold(model_config, X_train_fold, y_train_fold, X_val_fold, y_val_fold,
                        y_scaler_fold, n_epochs, patience=10):
    input_dim = X_train_fold.shape[1]
    model = BPNN(input_dim, model_config['hidden_dims'], dropout_rate=model_config['dropout_rate']).to(device)
    optimizer = optim.Adam(model.parameters(), lr=model_config['learning_rate'], weight_decay=model_config.get('weight_decay', 0))
    criterion = nn.MSELoss()

    train_dataset = TensorDataset(torch.FloatTensor(X_train_fold).to(device), torch.FloatTensor(y_train_fold).reshape(-1,1).to(device))
    train_loader = DataLoader(train_dataset, batch_size=model_config['batch_size'], shuffle=True)
    
    val_dataset = TensorDataset(torch.FloatTensor(X_val_fold).to(device), torch.FloatTensor(y_val_fold).reshape(-1,1).to(device))
    val_loader = DataLoader(val_dataset, batch_size=model_config['batch_size'], shuffle=False)

    epoch_train_mae_unnormalized = []
    epoch_val_mae_unnormalized = []
    
    best_val_loss_mae = float('inf')
    epochs_no_improve = 0
    best_model_state = None

    for epoch in range(n_epochs):
        model.train()
        # ... (training loop remains the same)
        fold_train_preds_unnorm = []
        fold_train_targets_unnorm = []
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            batch_preds_unnorm = y_scaler_fold.inverse_transform(outputs.detach().cpu().numpy())
            batch_targets_unnorm = y_scaler_fold.inverse_transform(batch_y.detach().cpu().numpy())
            fold_train_preds_unnorm.extend(batch_preds_unnorm.flatten())
            fold_train_targets_unnorm.extend(batch_targets_unnorm.flatten())
        current_train_mae_unnorm = mean_absolute_error(fold_train_targets_unnorm, fold_train_preds_unnorm)
        epoch_train_mae_unnormalized.append(current_train_mae_unnorm)

        model.eval()
        # ... (validation loop remains the same)
        fold_val_preds_unnorm = []
        fold_val_targets_unnorm = []
        with torch.no_grad():
            for batch_X_val, batch_y_val in val_loader:
                val_outputs = model(batch_X_val)
                batch_val_preds_unnorm = y_scaler_fold.inverse_transform(val_outputs.cpu().numpy())
                batch_val_targets_unnorm = y_scaler_fold.inverse_transform(batch_y_val.cpu().numpy())
                fold_val_preds_unnorm.extend(batch_val_preds_unnorm.flatten())
                fold_val_targets_unnorm.extend(batch_val_targets_unnorm.flatten())
        current_val_mae_unnorm = mean_absolute_error(fold_val_targets_unnorm, fold_val_preds_unnorm)
        epoch_val_mae_unnormalized.append(current_val_mae_unnorm)
        
        if current_val_mae_unnorm < best_val_loss_mae:
            best_val_loss_mae = current_val_mae_unnorm
            epochs_no_improve = 0
            best_model_state = model.state_dict()
        else:
            epochs_no_improve += 1
        
        if epochs_no_improve >= patience:
            break
            
    if best_model_state:
        model.load_state_dict(best_model_state)

    # Final evaluation on validation set for this fold
    model.eval()
    # ... (evaluation logic remains the same)
    all_y_pred_val_scaled = []
    all_y_val_scaled = []
    with torch.no_grad():
        for batch_X_val, batch_y_val in val_loader:
            val_outputs = model(batch_X_val)
            all_y_pred_val_scaled.extend(val_outputs.cpu().numpy())
            all_y_val_scaled.extend(batch_y_val.cpu().numpy())
    y_pred_val_unnorm = y_scaler_fold.inverse_transform(np.array(all_y_pred_val_scaled)).flatten()
    y_val_unnorm = y_scaler_fold.inverse_transform(np.array(all_y_val_scaled)).flatten()
    mae_val = mean_absolute_error(y_val_unnorm, y_pred_val_unnorm)
    rmse_val = np.sqrt(mean_squared_error(y_val_unnorm, y_pred_val_unnorm))
    r2_val = r2_score(y_val_unnorm, y_pred_val_unnorm)

    # Evaluation on training set
    all_y_pred_train_scaled = []
    all_y_train_scaled = []
    with torch.no_grad():
        for batch_X_train, batch_y_train in train_loader:
            train_outputs = model(batch_X_train)
            all_y_pred_train_scaled.extend(train_outputs.cpu().numpy())
            all_y_train_scaled.extend(batch_y_train.cpu().numpy())
    y_pred_train_unnorm = y_scaler_fold.inverse_transform(np.array(all_y_pred_train_scaled)).flatten()
    y_train_unnorm = y_scaler_fold.inverse_transform(np.array(all_y_train_scaled)).flatten()
    mae_train = mean_absolute_error(y_train_unnorm, y_pred_train_unnorm)
    rmse_train = np.sqrt(mean_squared_error(y_train_unnorm, y_pred_train_unnorm))
    r2_train = r2_score(y_train_unnorm, y_pred_train_unnorm)

    return {
        'mae_val': mae_val, 'rmse_val': rmse_val, 'r2_val': r2_val,
        'mae_train': mae_train, 'rmse_train': rmse_train, 'r2_train': r2_train,
        'train_loss_curve': epoch_train_mae_unnormalized,
        'val_loss_curve': epoch_val_mae_unnormalized,
        'model_state': best_model_state
    }


# --- 超参数定义 ---
param_grid = {
    'learning_rate': [0.0005, 0.001, 0.005],
    'hidden_dims': [[32], [64], [32, 16], [64, 32], [128, 64, 32]],
    'batch_size': [16, 32, 64],
    'dropout_rate': [0.2, 0.3, 0.4, 0.5],
    'weight_decay': [1e-4, 5e-4, 1e-3, 2e-3]
}
n_hyperparam_trials = 20
n_epochs_hyperparam_tuning = 50
patience_hyperparam = 5
n_epochs_final_training = 200
patience_final = 15

# --- 超参数调优 (在开发集上) ---
print("开始超参数调优...")
# ... (Hyperparameter tuning logic remains the same) ...
best_params = None
best_avg_val_mae = float('inf')
X_np = X_original_df.values
y_np = y_original_series.values
sampled_configs = []
for _ in range(n_hyperparam_trials):
    config = {}
    for key, values in param_grid.items():
        config[key] = random.choice(values)
    if config not in sampled_configs:
        sampled_configs.append(config)

for i, current_params in enumerate(sampled_configs):
    print(f"\n--- 超参数组合 {i+1}/{len(sampled_configs)} ---")
    print(current_params)
    kf_hyper = KFold(n_splits=3, shuffle=True, random_state=42)
    fold_val_maes = []
    for fold_idx, (train_idx, val_idx) in enumerate(kf_hyper.split(X_np, y_np)):
        X_train_fold, X_val_fold = X_np[train_idx], X_np[val_idx]
        y_train_fold, y_val_fold = y_np[train_idx], y_np[val_idx]
        x_scaler_fold = StandardScaler()
        X_train_fold_scaled = x_scaler_fold.fit_transform(X_train_fold)
        X_val_fold_scaled = x_scaler_fold.transform(X_val_fold)
        y_scaler_fold = StandardScaler()
        y_train_fold_scaled = y_scaler_fold.fit_transform(y_train_fold.reshape(-1, 1)).flatten()
        y_val_fold_scaled = y_scaler_fold.transform(y_val_fold.reshape(-1, 1)).flatten()
        fold_results = train_evaluate_fold(
            current_params, X_train_fold_scaled, y_train_fold_scaled,
            X_val_fold_scaled, y_val_fold_scaled, y_scaler_fold,
            n_epochs=n_epochs_hyperparam_tuning, patience=patience_hyperparam
        )
        fold_val_maes.append(fold_results['mae_val'])
    avg_fold_val_mae = np.mean(fold_val_maes)
    print(f"参数组 {i+1} 平均验证 MAE: {avg_fold_val_mae:.4f}")
    if avg_fold_val_mae < best_avg_val_mae:
        best_avg_val_mae = avg_fold_val_mae
        best_params = current_params

print("\n--- 超参数调优完成 ---")
if not best_params: best_params = sampled_configs[0]
print(f"最佳参数: {best_params}")
print(f"最佳平均验证 MAE: {best_avg_val_mae:.4f}")

# --- 使用最佳参数进行K折交叉验证评估 (在开发集上) ---
print("\n--- 使用最佳参数进行K折交叉验证评估 (在开发集内部) ---")
# ... (K-fold CV logic remains the same) ...
kf_final = KFold(n_splits=5, shuffle=True, random_state=123)
final_fold_metrics = []
for fold_num, (train_index, test_index) in enumerate(kf_final.split(X_np, y_np)):
    print(f"\nFold {fold_num + 1}/5")
    X_train, X_test = X_np[train_index], X_np[test_index]
    y_train, y_test = y_np[train_index], y_np[test_index]
    x_scaler = StandardScaler()
    X_train_scaled = x_scaler.fit_transform(X_train)
    X_test_scaled = x_scaler.transform(X_test)
    y_scaler = StandardScaler()
    y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1)).flatten()
    y_test_scaled = y_scaler.transform(y_test.reshape(-1, 1)).flatten()
    fold_results = train_evaluate_fold(
        best_params, X_train_scaled, y_train_scaled,
        X_test_scaled, y_test_scaled, y_scaler,
        n_epochs=n_epochs_final_training, patience=patience_final
    )
    final_fold_metrics.append(fold_results)
    print(f"  CV-Train MAE: {fold_results['mae_train']:.4f}, R2: {fold_results['r2_train']:.4f}")
    print(f"  CV-Validation MAE: {fold_results['mae_val']:.4f}, R2: {fold_results['r2_val']:.4f}")

# --- 计算交叉验证的平均性能 ---
avg_metrics = {
    'mae_train': np.mean([m['mae_train'] for m in final_fold_metrics]),
    'rmse_train': np.mean([m['rmse_train'] for m in final_fold_metrics]),
    'r2_train': np.mean([m['r2_train'] for m in final_fold_metrics]),
    'mae_val': np.mean([m['mae_val'] for m in final_fold_metrics]),
    'rmse_val': np.mean([m['rmse_val'] for m in final_fold_metrics]),
    'r2_val': np.mean([m['r2_val'] for m in final_fold_metrics]),
}

print("\nBPNN - Average CV Train Performance (on Development Set, Unnormalized):")
print(f"  MAE: {avg_metrics['mae_train']:.4f}, RMSE: {avg_metrics['rmse_train']:.4f}, R2 Score: {avg_metrics['r2_train']:.4f}")

print("\nBPNN - Average CV Validation Performance (on Development Set, Unnormalized):")
print(f"  MAE: {avg_metrics['mae_val']:.4f}, RMSE: {avg_metrics['rmse_val']:.4f}, R2 Score: {avg_metrics['r2_val']:.4f}")

# --- 训练最终的基准MLP模型 (在整个开发集上) ---
print("\n--- 训练最终基准MLP模型 (在整个80%开发集上) ---")
# 1. 创建并拟合最终的scalers
final_x_scaler = StandardScaler().fit(X_original_df.values)
final_y_scaler = StandardScaler().fit(y_original_series.values.reshape(-1, 1))

# 2. 缩放整个开发集
X_dev_scaled = final_x_scaler.transform(X_original_df.values)
y_dev_scaled = final_y_scaler.transform(y_original_series.values.reshape(-1, 1)).flatten()

# 3. 训练最终模型
# 为了使用早停，我们从开发集中临时划分一小部分作为验证集
X_final_train_s, X_final_val_s, y_final_train_s, y_final_val_s = train_test_split(
    X_dev_scaled, y_dev_scaled, test_size=0.1, random_state=42
)

final_model_results = train_evaluate_fold(
    best_params, X_final_train_s, y_final_train_s,
    X_final_val_s, y_final_val_s, final_y_scaler, # 传递final_y_scaler
    n_epochs=n_epochs_final_training, patience=patience_final
)
print("最终基准MLP模型训练完成。")

# ==============================================================================
# ======================== 新增的最终评估代码块开始 ========================
# ==============================================================================

print("\n--- 最终无偏评估 (在20%最终留出测试集上) ---")

# --- 1. 加载最终测试集数据 ---
try:
    final_test_df = pd.read_excel(final_test_data_path)
    print(f"最终测试集数据加载成功，形状: {final_test_df.shape}")
    X_final_test_df = final_test_df.drop(columns=[target_column_name])
    y_final_test_series = final_test_df[target_column_name]
except FileNotFoundError:
    print(f"错误: 最终测试集文件未找到: {final_test_data_path}")
    exit()
except Exception as e:
    print(f"加载最终测试集时发生错误: {e}")
    exit()

# --- 2. 加载最终模型并准备测试数据 ---
# 创建一个新的模型实例并加载最佳状态
final_model_for_test = BPNN(X_original_df.shape[1], best_params['hidden_dims'], dropout_rate=best_params['dropout_rate']).to(device)
final_model_for_test.load_state_dict(final_model_results['model_state'])
final_model_for_test.eval()

# 使用在整个开发集上fit的scaler来转换测试数据
X_final_test_scaled = final_x_scaler.transform(X_final_test_df.values)
X_final_test_tensor = torch.FloatTensor(X_final_test_scaled).to(device)

# --- 3. 进行预测并反归一化 ---
with torch.no_grad():
    y_pred_final_test_scaled_tensor = final_model_for_test(X_final_test_tensor)

y_pred_final_test_scaled_np = y_pred_final_test_scaled_tensor.cpu().numpy()
y_pred_final_test_unnorm = final_y_scaler.inverse_transform(y_pred_final_test_scaled_np).flatten()

# --- 4. 计算并打印最终性能指标 ---
mae_final_bpnn = mean_absolute_error(y_final_test_series.values, y_pred_final_test_unnorm)
rmse_final_bpnn = np.sqrt(mean_squared_error(y_final_test_series.values, y_pred_final_test_unnorm))
r2_final_bpnn = r2_score(y_final_test_series.values, y_pred_final_test_unnorm)

print("\n--- 最终BPNN模型在最终测试集上的性能 ---")
print(f"MAE: {mae_final_bpnn:.4f}")
print(f"RMSE: {rmse_final_bpnn:.4f}")
print(f"R2 Score: {r2_final_bpnn:.4f}")

# (可选) 绘制真实值 vs 预测值图
plt.figure(figsize=(8, 8))
plt.scatter(y_final_test_series.values, y_pred_final_test_unnorm, alpha=0.7, edgecolors='w', linewidth=0.5, color='green')
min_val = min(y_final_test_series.min(), y_pred_final_test_unnorm.min())
max_val = max(y_final_test_series.max(), y_pred_final_test_unnorm.max())
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2)
plt.xlabel('Actual Rowing Distance')
plt.ylabel('Predicted Rowing Distance (MLP)')
plt.title('MLP Baseline Model: Actual vs. Predicted (Hold-Out Test Set)')
plt.grid(True, linestyle='--', alpha=0.7)
plot_filename_actual_vs_pred_mlp = os.path.join(output_plot_path, "mlp_baseline_model_actual_vs_predicted.png")
try:
    plt.savefig(plot_filename_actual_vs_pred_mlp, dpi=300, bbox_inches='tight')
    print(f"\nMLP基准模型真实值 vs 预测值图已保存到: {plot_filename_actual_vs_pred_mlp}")
except Exception as e:
    print(f"保存MLP基准模型真实值 vs 预测值图时发生错误: {e}")
plt.show()

# ==============================================================================
# ========================= 新增的最终评估代码块结束 =========================
# ==============================================================================

# 原有的图表绘制部分可以继续保留，它们是基于CV结果的
# ... (此处省略了原有图表绘制代码，逻辑不变) ...


In [ ]:
# 设置目标保存路径
target_path = r"C:\Users\Michael Wang\OneDrive\小论文\毕业论文改写\WGAN-GP\插图\返修插图"

# 确保目录存在，如果不存在则创建
import os
os.makedirs(target_path, exist_ok=True)

# 1. 绘制评估指标图 (MAE 和 R2)
metrics_plot_names_en = ['MAE', 'R2 Score']
values_train_plot = [avg_metrics['mae_train'], avg_metrics['r2_train']]
values_test_plot = [avg_metrics['mae_val'], avg_metrics['r2_val']]
x_axis_plot = np.arange(len(metrics_plot_names_en))
plt.figure(figsize=(10, 6))
plt.bar(x_axis_plot - 0.2, values_train_plot, width=0.4, label='CV Train', align='center')
plt.bar(x_axis_plot + 0.2, values_test_plot, width=0.4, label='CV Validation', align='center')
plt.xticks(x_axis_plot, metrics_plot_names_en)
plt.ylabel('Score')
plt.title('BPNN Average CV Evaluation Metrics (Original Data)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)

# 修改保存路径和文件名，添加前缀
plot_filename_metrics = os.path.join(target_path, "MLP原始数据_bpnn_average_evaluation_metrics.png")
plt.savefig(plot_filename_metrics, dpi=300, bbox_inches='tight')
plt.show()

# 2. 绘制每折的训练和测试损失 (MAE, 反归一化)
all_folds_train_loss_curves = [m['train_loss_curve'] for m in final_fold_metrics]
all_folds_val_loss_curves = [m['val_loss_curve'] for m in final_fold_metrics]
plt.figure(figsize=(14, 7))
max_epochs_ran = 0
for i in range(len(all_folds_train_loss_curves)):
    epochs_this_fold = len(all_folds_train_loss_curves[i])
    max_epochs_ran = max(max_epochs_ran, epochs_this_fold)
    plt.plot(range(1, epochs_this_fold + 1), all_folds_train_loss_curves[i], label=f'CV Train Fold {i+1}', linestyle='-')
    plt.plot(range(1, epochs_this_fold + 1), all_folds_val_loss_curves[i], label=f'CV Validation Fold {i+1}', linestyle='--')
plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error (MAE) - Unnormalized')
plt.title('BPNN Training and CV Validation MAE (Unnormalized) per Fold')
if max_epochs_ran > 0: plt.xlim(1, max_epochs_ran)
plt.legend(loc='upper right')
plt.grid(True, linestyle='--', alpha=0.7)

# 修改保存路径和文件名，添加前缀
plot_filename_loss = os.path.join(target_path, "MLP原始数据_bpnn_mae_loss_per_fold_unnormalized.png")
plt.savefig(plot_filename_loss, dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# (可选) 绘制真实值 vs 预测值图
plt.figure(figsize=(8, 8))
plt.scatter(y_final_test_series.values, y_pred_final_test_unnorm, alpha=0.7, edgecolors='w', linewidth=0.5)
min_val = min(y_final_test_series.min(), y_pred_final_test_unnorm.min())
max_val = max(y_final_test_series.max(), y_pred_final_test_unnorm.max())
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2)
plt.xlabel('Actual Rowing Distance')
plt.ylabel('Predicted Rowing Distance (MLP)')
plt.title('MLP Baseline Model: Actual vs. Predicted (Hold-Out Test Set)')
plt.grid(True, linestyle='--', alpha=0.7)
plot_filename_actual_vs_pred_mlp = os.path.join(output_plot_path, "mlp_baseline_model_actual_vs_predicted.png")
try:
    plt.savefig(plot_filename_actual_vs_pred_mlp, dpi=300, bbox_inches='tight')
    print(f"\nMLP基准模型真实值 vs 预测值图已保存到: {plot_filename_actual_vs_pred_mlp}")
except Exception as e:
    print(f"保存MLP基准模型真实值 vs 预测值图时发生错误: {e}")
plt.show()
# 修改保存路径和文件名，添加前缀
plot_filename_loss = os.path.join(target_path, "MLP原始数据_mlp_baseline_model_actual_vs_predicted.png")
plt.savefig(plot_filename_loss, dpi=300, bbox_inches='tight')